In [1]:
import timm
import torch.nn as nn

class DualInputSwinTransformer(nn.Module):
    def __init__(self, num_classes):
        super(DualInputSwinTransformer, self).__init__()
        # 定义两个独立的Swin Transformer模型作为编码器
        self.swin_gaussian = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True, num_classes=0)
        self.swin_unet = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True, num_classes=0)
        
        # 定义一个分类器，将两个编码器的输出合并后进行分类
        self.classifier = nn.Sequential(
            nn.Linear(768 * 2, 512),  # 假设每个Swin Transformer的输出特征大小为768
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x1, x2 = x
        x1 = self.swin_gaussian(x1)
        x2 = self.swin_unet(x2)
        x = torch.cat((x1, x2), dim=1)  # 在特征维度上合并两个编码器的输出
        x = self.classifier(x)
        return x

model = DualInputSwinTransformer(num_classes=100)

# 打印所有层的名称
print("所有层的名称:")
for name, module in model.named_modules():
    print(name)

# 找到并打印最后两层的名称
layer_names = [name for name, _ in model.named_modules()]
if len(layer_names) >= 2:
    print("\n最后两层的名称:")
    print(layer_names[-2:])
else:
    print("模型中没有足够的层来显示最后两层的名称")

model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

所有层的名称:

swin_gaussian
swin_gaussian.patch_embed
swin_gaussian.patch_embed.proj
swin_gaussian.patch_embed.norm
swin_gaussian.layers
swin_gaussian.layers.0
swin_gaussian.layers.0.downsample
swin_gaussian.layers.0.blocks
swin_gaussian.layers.0.blocks.0
swin_gaussian.layers.0.blocks.0.norm1
swin_gaussian.layers.0.blocks.0.attn
swin_gaussian.layers.0.blocks.0.attn.qkv
swin_gaussian.layers.0.blocks.0.attn.attn_drop
swin_gaussian.layers.0.blocks.0.attn.proj
swin_gaussian.layers.0.blocks.0.attn.proj_drop
swin_gaussian.layers.0.blocks.0.attn.softmax
swin_gaussian.layers.0.blocks.0.drop_path1
swin_gaussian.layers.0.blocks.0.norm2
swin_gaussian.layers.0.blocks.0.mlp
swin_gaussian.layers.0.blocks.0.mlp.fc1
swin_gaussian.layers.0.blocks.0.mlp.act
swin_gaussian.layers.0.blocks.0.mlp.drop1
swin_gaussian.layers.0.blocks.0.mlp.norm
swin_gaussian.layers.0.blocks.0.mlp.fc2
swin_gaussian.layers.0.blocks.0.mlp.drop2
swin_gaussian.layers.0.blocks.0.drop_path2
swin_gaussian.layers.0.blocks.1
swin_gaussian.l

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import os
import timm
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder

class DualNPYDataset(Dataset):
    def __init__(self, csv_file, root_dir_gaussian, root_dir_unet, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir_gaussian = root_dir_gaussian
        self.root_dir_unet = root_dir_unet
        self.transform = transform

        # 初始化LabelEncoder并对标签进行编码
        self.label_encoder = LabelEncoder()
        # 假设标签在CSV文件的第三列
        self.encoded_labels = self.label_encoder.fit_transform(self.annotations.iloc[:, 2].values)

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        # 加载图像...
        img_name_gaussian = os.path.join(self.root_dir_gaussian, str(self.annotations.iloc[index, -1]) + '.npy')
        image_gaussian = np.load(img_name_gaussian).astype(np.float32)
        if self.transform:
            image_gaussian = self.transform(image_gaussian)
        
        img_name_unet = os.path.join(self.root_dir_unet, str(self.annotations.iloc[index, -1]) + '.npy')
        image_unet = np.load(img_name_unet).astype(np.float32)
        if self.transform:
            image_unet = self.transform(image_unet)

        # 使用已编码的标签
        label = self.encoded_labels[index]
        label = torch.tensor(label, dtype=torch.long)  # 转换为Tensor
        return (image_gaussian, image_unet), label
    
# 定义图像转换
transform = transforms.Compose([
    transforms.ToTensor(),  # 将numpy数组转换为torch张量，并且从(H, W, C)转换为(C, H, W)且归一化到[0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 归一化
])

class DualInputSwinTransformer(nn.Module):
    def __init__(self, num_classes):
        super(DualInputSwinTransformer, self).__init__()
        # 定义两个独立的Swin Transformer模型作为编码器
        self.swin_gaussian = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True, num_classes=0)
        self.swin_unet = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True, num_classes=0)
        
        # 定义一个分类器，将两个编码器的输出合并后进行分类
        self.classifier = nn.Sequential(
            nn.Linear(768 * 2, 512),  # 假设每个Swin Transformer的输出特征大小为768
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x1, x2 = x
        x1 = self.swin_gaussian(x1)
        x2 = self.swin_unet(x2)
        # 应用 6:4 的权重比例
        weighted_x1 = x1 * 0.6
        weighted_x2 = x2 * 0.4
        # 在特征维度上合并加权的编码器输出
        x = torch.cat((weighted_x1, weighted_x2), dim=1)
        return x  # 返回特征而不是分类器的输出

# 训练和评估参数
num_epochs = 15
batch_size = 32
learning_rate = 0.001

# 设备配置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 初始化数据集
train_dataset = DualNPYDataset(csv_file='/kaggle/input/double-chanel-transformer/train.csv',
                               root_dir_gaussian='/kaggle/input/double-chanel-transformer/transformer_gaus_train/transformer_gaus_train',
                               root_dir_unet='/kaggle/input/double-chanel-transformer/transformer_predicted_train/transformer_predicted_train',
                               transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

validation_dataset = DualNPYDataset(csv_file='/kaggle/input/double-chanel-transformer/validation.csv',
                               root_dir_gaussian='/kaggle/input/double-chanel-transformer/transformer_gaus_validation/transformer_gaus_validation',
                               root_dir_unet='/kaggle/input/double-chanel-transformer/transformer_predicted_validation/transformer_predicted_validation',
                               transform=transform)

validation_loader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle=False)
test_dataset = DualNPYDataset(csv_file='/kaggle/input/double-chanel-transformer/test.csv',
                               root_dir_gaussian='/kaggle/input/double-chanel-transformer/transformer_gaus_test/transformer_gaus_test',
                               root_dir_unet='/kaggle/input/double-chanel-transformer/transformer_predicted_test/transformer_predicted_test',
                               transform=transform)

test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# 模型初始化
num_classes = len(np.unique(train_dataset.annotations['labels']))
model = DualInputSwinTransformer(num_classes=num_classes).to(device)

# 创建一个权重数组，给第三类更高的权重
weights = torch.tensor([1.0, 1.0, 1.0, 1.0], dtype=torch.float32).to(device)

# 使用加权损失函数
criterion = nn.CrossEntropyLoss(weight=weights)

optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-2)

# 学习率调度器
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

In [9]:
def extract_features(loader, model, device):
    model.eval()
    features = []
    labels = []

    with torch.no_grad():
        for (image_gaussian, image_unet), targets in loader:
            image_gaussian, image_unet = image_gaussian.to(device), image_unet.to(device)
            # 提取特征
            output = model((image_gaussian, image_unet))
            features.append(output.cpu().numpy())
            labels.append(targets.cpu().numpy())

    features = np.concatenate(features)
    labels = np.concatenate(labels)
    return features, labels

In [10]:
def train_model(model, train_loader, validation_loader, test_loader, criterion, optimizer, scheduler, num_epochs=15, device='cuda'):
    model.to(device)
    # 解冻特定层的参数
    for name, param in model.named_parameters():
        if 'classifier' in name:
            param.requires_grad = True
        else:
            param.requires_grad = False

    # 确保优化器仅更新requires_grad=True的参数
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        for (images_gaussian, images_unet), labels in train_loader:
            images_gaussian = images_gaussian.to(device)
            images_unet = images_unet.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model((images_gaussian, images_unet))
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        
        scheduler.step()

        # 在每个epoch结束后评估模型性能
        train_accuracy = evaluate_model(model, train_loader, device)
        validation_accuracy = evaluate_model(model, validation_loader, device)
        test_accuracy = evaluate_model(model, test_loader, device)

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, '
              f'Train Accuracy: {train_accuracy:.2f}%, '
              f'Validation Accuracy: {validation_accuracy:.2f}%, '
              f'Test Accuracy: {test_accuracy:.2f}%')

def evaluate_model(model, data_loader, device='cuda'):
    model.eval()
    total = correct = 0
    with torch.no_grad():
        for (images_gaussian, images_unet), labels in data_loader:
            images_gaussian = images_gaussian.to(device)
            images_unet = images_unet.to(device)
            labels = labels.to(device)

            outputs = model((images_gaussian, images_unet))
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    return accuracy


In [12]:
model_path = '/kaggle/working/trained_model.pth'

In [1]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# 提取特征
train_features, train_labels = extract_features(train_loader, model, device)
validation_features, validation_labels = extract_features(validation_loader, model, device)
test_features, test_labels = extract_features(test_loader, model, device)

# 训练 SVM
svm_clf = SVC(kernel='linear', C=1, probability=True)  # 设置 probability=True 以输出概率
svm_clf.fit(train_features, train_labels)

# 验证 SVM
validation_predictions_svm = svm_clf.predict(validation_features)
validation_accuracy_svm = accuracy_score(validation_labels, validation_predictions_svm)
print(f'SVM 验证准确率: {validation_accuracy_svm:.4f}')

# 测试 SVM
test_predictions_svm = svm_clf.predict(test_features)
test_accuracy_svm = accuracy_score(test_labels, test_predictions_svm)
print(f'SVM 测试准确率: {test_accuracy_svm:.4f}')

SVM 验证准确率: 0.944
SVM 测试准确率: 0.953


In [2]:
from sklearn.neural_network import MLPClassifier

# 训练 MLP
mlp_clf = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)
mlp_clf.fit(train_features, train_labels)

# 验证 MLP
validation_predictions_mlp = mlp_clf.predict(validation_features)
validation_accuracy_mlp = accuracy_score(validation_labels, validation_predictions_mlp)
print(f'MLP 验证准确率: {validation_accuracy_mlp:.4f}')

# 测试 MLP
test_predictions_mlp = mlp_clf.predict(test_features)
test_accuracy_mlp = accuracy_score(test_labels, test_predictions_mlp)
print(f'MLP 测试准确率: {test_accuracy_mlp:.4f}')

MLP 验证准确率: 0.945
MLP 测试准确率: 0.937


In [3]:
import xgboost as xgb

# 转换数据为 DMatrix 格式
dtrain = xgb.DMatrix(train_features, label=train_labels)
dvalid = xgb.DMatrix(validation_features, label=validation_labels)
dtest = xgb.DMatrix(test_features, label=test_labels)

# 设置参数
params = {
    'objective': 'multi:softprob',  # 用于多分类
    'num_class': num_classes,
    'eval_metric': 'mlogloss',
    'max_depth': 6,
    'eta': 0.1,
    'seed': 42
}

# 训练 XGBoost
xgb_clf = xgb.train(params, dtrain, num_boost_round=100, evals=[(dvalid, 'validation')])

# 验证 XGBoost
validation_predictions_xgb = xgb_clf.predict(dvalid)
validation_accuracy_xgb = accuracy_score(validation_labels, np.argmax(validation_predictions_xgb, axis=1))
print(f'XGBoost 验证准确率: {validation_accuracy_xgb:.4f}')

# 测试 XGBoost
test_predictions_xgb = xgb_clf.predict(dtest)
test_accuracy_xgb = accuracy_score(test_labels, np.argmax(test_predictions_xgb, axis=1))
print(f'XGBoost 测试准确率: {test_accuracy_xgb:.4f}')

XGB 验证准确率: 0.942
XGB 测试准确率: 0.936


In [4]:
from sklearn.ensemble import VotingClassifier

# 创建 VotingClassifier
voting_clf = VotingClassifier(
    estimators=[
        ('svm', svm_clf),
        ('mlp', mlp_clf),
        ('xgb', xgb.XGBClassifier(objective='multi:softprob', n_estimators=100, max_depth=6, learning_rate=0.1))
    ],
    voting='soft',  # 使用软投票
    weights=[1, 1, 1]  # 根据需要调整权重
)

# 训练 VotingClassifier
voting_clf.fit(train_features, train_labels)

# 在测试集上进行预测
test_predictions_voting = voting_clf.predict(test_features)

# 计算最终准确率
voting_accuracy = accuracy_score(test_labels, test_predictions_voting)
print(f'Voting Classifier 测试准确率: {voting_accuracy:.4f}')

Voting Classifier 测试准确率: 0.954
